Test 1 - 5/22 Night:
- Inital Scrape via API works but content for email isn't showing
- Trying nltk, spacy and beautifulsoup for word processing to see if I can get more accurate scraping -> 2nd rendition doesn't process as many emails as inital scrape

In [1]:
#!pip install python-dotenv pandas openpyxl nltk spacy
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
import os.path
import base64
import json
import re
import pandas as pd
import nltk
import spacy

from bs4 import BeautifulSoup
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [2]:
# initializing spaCy
nlp = spacy.load('en_core_web_sm')

In [3]:
SCOPES = [
    'https://www.googleapis.com/auth/gmail.readonly',
    'https://www.googleapis.com/auth/userinfo.email',
    'https://www.googleapis.com/auth/gmail.addons.current.action.compose',
    'https://www.googleapis.com/auth/gmail.addons.current.message.action',
    'openid'
]


def main():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels and processes messages.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
        print("Loaded credentials from token.json")
    else:
        print("token.json not found, need to authenticate")
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                creds.refresh(Request())
                print("Credentials refreshed")
            except Exception as e:
                print(f"Failed to refresh credentials: {e}")
        else:
            try:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'credentials.json', SCOPES)
                creds = flow.run_local_server(port=0)
                print("Credentials obtained from flow")
            except Exception as e:
                print(f"Failed to obtain credentials: {e}")
                return
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
            print("Credentials saved to token.json")

    try:
        # Call the Gmail API
        service = build('gmail', 'v1', credentials=creds)

        # Get list of messages from the last 5 months
        results = service.users().messages().list(userId='me', q='newer_than:150d').execute()
        messages = results.get('messages', [])

        job_data = []

        if not messages:
            print('No messages found.')
        else:
            print('Messages:')
            for message in messages:
                msg = service.users().messages().get(userId='me', id=message['id']).execute()
                payload = msg['payload']
                headers = payload.get('headers')
                for header in headers:
                    if header['name'] == 'Subject':
                        subject = header['value']
                    if header['name'] == 'From':
                        from_ = header['value']
                    if header['name'] == 'Date':
                        date_ = header['value']
                job_data.append({
                    "Subject": subject,
                    "From": from_,
                    "Date": date_
                })
                print(f"Processed email from {from_} with subject: {subject}")

        # Create a DataFrame
        df = pd.DataFrame(job_data)

        # Add placeholder columns for additional information
        df["Company"] = ""
        df["Position"] = ""
        df["Status"] = ""
        df["Notes"] = ""

        # Save the data to an Excel file
        excel_filename = "job_tracker.xlsx"

        # Create a new Excel file with a worksheet
        df.to_excel(excel_filename, index=False)
        print(f"Job tracker saved to {excel_filename}")

    except HttpError as error:
        print(f"An error occurred: {error}")

if __name__ == '__main__':
    main()

Loaded credentials from token.json
Messages:
Processed email from Google Developer Program <googledev-noreply@google.com> with subject: Welcome to the Google Developer Program
Processed email from Google <no-reply@accounts.google.com> with subject: Security alert
Processed email from no-reply@optimove.com with subject: Thank you for applying to Optimove
Processed email from urban@myworkday.com with subject: Your application has been received
Processed email from urban@myworkday.com with subject: Verify your candidate account
Processed email from TikTok <noreply@careers.tiktok.com> with subject: Thank you for applying to TikTok!
Processed email from LinkedIn <jobs-noreply@linkedin.com> with subject: Arya, your application was sent to Degas
Processed email from Arya Amarnath <realyx.arya@gmail.com> with subject: Re: Help with TS-SCI Clearance
Processed email from David Kasten <dlkasten@gmail.com> with subject: Re: Help with TS-SCI Clearance
Processed email from Dice <dice@connect.dice.co

In [4]:
# function to extract job info from email
def extract_job_info(subject, body):
    company = None
    role_title = None
    salary_rate = None
    link_to_job_advert = None
    application_date = None
    contact_name = None
    contact_email = None
    contact_tel = None

    doc = nlp(body)

    # extract entities using NLP
    for ent in doc.ents:
        if ent.label_ == 'ORG':
            company = ent.text
        elif ent.label_ == 'MONEY':
            salary_rate = ent.text
        elif ent.label_ in ('DATE', 'TIME'):
            application_date = ent.text
        elif ent.label_ == 'PERSON':
            contact_name = ent.text

    # fallback extraction using regex
    if not company:
        company_match = re.search(r'Company: (.+)', body)
        if company_match:
            company = company_match.group(1)
    if not role_title:
        role_title_match = re.search(r'Position: (.+)', body)
        if role_title_match:
            role_title = role_title_match.group(1)
    if not salary_rate:
        salary_rate_match = re.search(r'Salary: (.+)', body)
        if salary_rate_match:
            salary_rate = salary_rate_match.group(1)
    if not link_to_job_advert:
        link_to_job_advert_match = re.search(r'Link: (http.+)', body)
        if link_to_job_advert_match:
            link_to_job_advert = link_to_job_advert_match.group(1)
    if not application_date:
        application_date_match = re.search(r'Date: (\d{2}/\d{2}/\d{2})', body)
        if application_date_match:
            application_date = application_date_match.group(1)
    if not contact_name or not contact_email or not contact_tel:
        contact_match = re.search(r'Contact: (.+)', body)
        if contact_match:
            contact_info = contact_match.group(1).split(' - ')
            if len(contact_info) >= 3:
                contact_name, contact_email, contact_tel = contact_info[:3]

    return {
        "Company": company,
        "Role Title": role_title,
        "Salary/Rate": salary_rate,
        "Link to Job Advert": link_to_job_advert,
        "Application Date": application_date,
        "Contact Name": contact_name,
        "Contact Email": contact_email,
        "Contact Tel": contact_tel
    }

# function to get the body of the email
def get_body(payload):
    body = ""
    if 'parts' in payload:
        for part in payload['parts']:
            if part['mimeType'] == 'text/plain':
                body += base64.urlsafe_b64decode(part['body']['data']).decode('utf-8')
            elif part['mimeType'] == 'text/html':
                body += BeautifulSoup(base64.urlsafe_b64decode(part['body']['data']), 'html.parser').get_text()
    else:
        body = base64.urlsafe_b64decode(payload['body']['data']).decode('utf-8')
    return body

# main function to fetch and process emails
def main():
    creds = None
    # load credentials
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
        print("loaded credentials from token.json")
    else:
        print("token.json not found, need to authenticate")
    
    # refresh or get new credentials if necessary
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                creds.refresh(Request())
                print("credentials refreshed")
            except Exception as e:
                print(f"failed to refresh credentials: {e}")
        else:
            try:
                flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
                creds = flow.run_local_server(port=0)
                print("credentials obtained from flow")
            except Exception as e:
                print(f"failed to obtain credentials: {e}")
                return
        # save credentials
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
            print("credentials saved to token.json")

    try:
        # call the Gmail API
        service = build('gmail', 'v1', credentials=creds)

        # get list of messages with specific keywords from the last 5 months
        query = 'subject:(job application OR applied OR application received)'
        results = service.users().messages().list(userId='me', q=query).execute()
        messages = results.get('messages', [])

        job_data = []

        if not messages:
            print('no messages found')
        else:
            print('messages:')
            for message in messages:
                msg = service.users().messages().get(userId='me', id=message['id']).execute()
                payload = msg['payload']
                headers = payload.get('headers')
                subject = None
                for header in headers:
                    if header['name'] == 'Subject':
                        subject = header['value']
                body = get_body(payload)
                job_info = extract_job_info(subject, body)
                job_data.append(job_info)
                print(f"processed email with subject: {subject}")

        # create a DataFrame
        df = pd.DataFrame(job_data)

        # add placeholder columns for additional information
        df["Company"] = df["Company"].fillna("")
        df["Role Title"] = df["Role Title"].fillna("")
        df["Salary/Rate"] = df["Salary/Rate"].fillna("")
        df["Link to Job Advert"] = df["Link to Job Advert"].fillna("")
        df["Application Date"] = df["Application Date"].fillna("")
        df["Contact Name"] = df["Contact Name"].fillna("")
        df["Contact Email"] = df["Contact Email"].fillna("")
        df["Contact Tel"] = df["Contact Tel"].fillna("")

        # save the data to an Excel file
        excel_filename = "job_tracker_v2.xlsx"

        # create a new Excel file with a worksheet
        df.to_excel(excel_filename, index=False)
        print(f"job tracker saved to {excel_filename}")

    except HttpError as error:
        print(f"an error occurred: {error}")

if __name__ == '__main__':
    main()

loaded credentials from token.json
messages:
processed email with subject: Thank you - we've received your job application
job tracker saved to job_tracker_v2.xlsx
